In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from collections import defaultdict
import os
import seaborn as sns
import scipy 
from functools import reduce
import sys


file_dirname = os.path.dirname(os.path.abspath('__file__'))
sys.path.insert(1,os.path.join(file_dirname,'..','statistics'))




In [2]:
from module_analysis_transposon_sites import (frequency_transposons,reads_per_transposon, transposon_density,
                                            median_feature_essentials,median_feature_nonessentials,filter_low_and_biased_reads_genes,
                                            reads_distribution_per_gene)

In [3]:
#%% Import of dataframes output from the SATAY pipeline


names_libraries={'wt':'data_wt_merged.xlsx','dbem1dbem3dnrp1':'data-enzo-dbem13dnrp1.xlsx'}
data_library=[]
for i in names_libraries.keys():
 data_library.append(pd.read_excel(file_dirname+'/../../datasets/'+names_libraries[i],index_col='Unnamed: 0',engine='openpyxl'))


In [5]:
#%% Removing the ADE2 and URA3 gene reads and insertions from the population

for i in np.arange(0,len(data_library)):
    data_library[i]=data_library[i][data_library[i].Standard_name != 'ADE2']
    data_library[i]=data_library[i][data_library[i].Standard_name != 'URA3']

data_library_pd=pd.concat(data_library,keys=names_libraries.keys(),sort=True)
data_library_pd.fillna(0,inplace=True)

In [6]:

measures_satay=defaultdict(dict)

for keys in names_libraries.keys():
    measures_satay[keys]["bp_per_tr"]=frequency_transposons(data_library_pd.loc[keys])
    measures_satay[keys]["reads_per_tr"]=reads_per_transposon(data_library_pd.loc[keys])
    measures_satay[keys]["tr_pr_100bp"]=100*transposon_density(data_library_pd.loc[keys])
    measures_satay[keys]["tr_essentials_median"]=median_feature_essentials(data_library_pd.loc[keys],feature="Ninsertions")
    measures_satay[keys]["tr_nonessentials_median"]=median_feature_nonessentials(data_library_pd.loc[keys],feature="Ninsertions")
    measures_satay[keys]["reads_per_tr_essentials"]=median_feature_essentials(data_library_pd.loc[keys],feature="Nreadsperinsrt")
    measures_satay[keys]["reads_per_tr_nonessentials"]=median_feature_nonessentials(data_library_pd.loc[keys],feature="Nreadsperinsrt")

measures_satay_pd=pd.DataFrame(measures_satay)
measures_satay_pd

There are, in average 4.1387178564499365 transposons per each 100 basepairs
There are, in average 3.524039374568914 transposons per each 100 basepairs


,wt,dbem1dbem3dnrp1
bp_per_tr,24.162072,28.376527
reads_per_tr,75.231884,8.096154
tr_pr_100bp,4.138718,3.524039
tr_essentials_median,18.000000,12.000000
tr_nonessentials_median,24.000000,21.000000
reads_per_tr_essentials,41.709330,4.860390
reads_per_tr_nonessentials,79.500000,8.490196


In [7]:
data_wt=data_library_pd.loc['wt'].copy()
data_mutant=data_library_pd.loc['dbem1dbem3dnrp1'].copy()

data_nrp1=data_mutant
####### Transposon density vs genes ####################

data_nrp1['tr-density']=data_nrp1['Ninsertions']/data_nrp1['Nbasepairs']
data_wt['tr-density']=data_wt['Ninsertions']/data_wt['Nbasepairs']
data_nrp1['reads-density']=data_nrp1['Nreads']/data_nrp1['Nbasepairs']
data_wt['reads-density']=data_wt['Nreads']/data_wt['Nbasepairs']


In [7]:
### Filtering data to compute median_feature_nonessentials

data_filter_wt=filter_low_and_biased_reads_genes(data_wt)
data_filter_dnrp1=filter_low_and_biased_reads_genes(data_nrp1)

data_filter_dnrp1.fillna(0,inplace=True)
data_filter_wt.fillna(0,inplace=True)

data_filter_wt.index=data_wt['Standard_name']
data_filter_dnrp1.index=data_nrp1["Standard_name"]

In [ ]:
#%% Plot transposon density (fig 1B Benoit) highlighting the centromere position

#data_wt.index=data_wt['Standard_name']

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
ax.plot(data_wt['tr-density'],'.',alpha=0.5,color='b')
ax.set_ylabel('transposon density: tn/bp')
ax.set_xlabel('genes')

## annotated centromeres
for i in  data_wt.index:

    if type(data_wt.loc[i,'Feature_type'])!=str:
        
        if data_wt.loc[i,'Feature_type'].any() =='Centromere': 
            
            ax.vlines(x=i,ymin=0,ymax=0.8,linestyles='--',alpha=0.3)
            ax.text(x=i,y=0.6,s='centromere',rotation=90,fontsize=8)
    else:

         if data_wt.loc[i,'Feature_type'] =='Centromere': 
            
            ax.vlines(x=i,ymin=0,ymax=0.8,linestyles='--',alpha=0.3)
            ax.text(x=i,y=0.6,s='centromere',rotation=90,fontsize=8)
    

In [ ]:
#%% Transposon density for comparing two libraries
fig=plt.figure(figsize=(10,9))
grid = plt.GridSpec(2, 1, wspace=0.0, hspace=0.0)
ax = plt.subplot(grid[0,0])
ax2 = plt.subplot(grid[1,0]) 

ax.set_xticks([])
# for minor ticks
ax.set_xticks([], minor=True)

ax.plot(data_wt['tr-density'],".",alpha=0.5,color='b')
ax.set_ylabel('transposon density: tn/bp')

## annotated centromeres
for i in data_wt.index:
    
    if type(data_wt.loc[i,'Feature_type'])!=str:
        
        if data_wt.loc[i,'Feature_type'].any() =='Centromere': 
            
            ax.vlines(x=i,ymin=0,ymax=0.8,linestyles='--',alpha=0.3)
            ax.text(x=i,y=0.6,s='centromere',rotation=90,fontsize=8)
    else:

         if data_wt.loc[i,'Feature_type'] =='Centromere': 
            
            ax.vlines(x=i,ymin=0,ymax=0.8,linestyles='--',alpha=0.3)
            ax.text(x=i,y=0.6,s='centromere',rotation=90,fontsize=8)

ax2.plot(data_nrp1['tr-density'],".",alpha=0.5,color='orange')
ax2.set_ylabel('transposon density: tn/bp mutant ')
ax2.set_xlabel('genes')

ax2.set_xticks([])
# for minor ticks
ax2.set_xticks([], minor=True)
## annotated centromeres
for i in data_nrp1.index:
    
    if type(data_nrp1.loc[i,'Feature_type'])!=str:
        
        if data_nrp1.loc[i,'Feature_type'].any() =='Centromere': 
            
            ax2.vlines(x=i,ymin=0,ymax=0.8,linestyles='--',alpha=0.3)
            ax2.text(x=i,y=0.6,s='centromere',rotation=90,fontsize=8)
    else:

         if data_nrp1.loc[i,'Feature_type'] =='Centromere': 
            
            ax2.vlines(x=i,ymin=0,ymax=0.8,linestyles='--',alpha=0.3)
            ax2.text(x=i,y=0.6,s='centromere',rotation=90,fontsize=8)


##  Plot reads per transposon  highlighting the centromere position

In [ ]:
strain=data_nrp1

fig=plt.figure(figsize=(15,15))
grid = plt.GridSpec(4, 1, wspace=0.0, hspace=0.1)

ax = plt.subplot(grid[0,0])
ax2 = plt.subplot(grid[2,0])   
ax3 = plt.subplot(grid[3,0]) 
ax4 = plt.subplot(grid[1,0])  

ax.set_title('Variation along the genome for dnrp1 merged dataset')
ax.plot(strain['reads-per-tr'],".",alpha=0.9,color='green')
ax.set_ylabel('reads per tr filtered for high tr-density')

ax.set_ylim(0,2000)

ax4.plot(strain['Nreadsperinsrt'],".",alpha=0.5,color='green')
ax4.set_ylabel('reads per tr without density filter')
ax4.set_ylim(0,2000)
     

ax2.plot(strain['tr-density'],".",alpha=0.7,color='black')
ax2.set_ylabel('transposon density')
ax2.set_ylim(0,1)

ax3.plot(strain['reads-density'], ".",alpha=0.5,color='b')
ax3.set_ylabel('Reads density')
ax3.set_xlabel('genomic regions')
ax3.set_ylim(0,250)


In [ ]:
#%% Compare the fold change of the reads per transposon per library

fold_change=data_wt['reads-per-tr']/data_nrp1['reads-per-tr']
fold_change.replace([np.inf, -np.inf], np.nan, inplace=True)
fold_change.fillna(0,inplace=True)

fold_change_nrp1=data_nrp1['reads-per-tr']/data_wt['reads-per-tr']
fold_change_nrp1.replace([np.inf, -np.inf], np.nan, inplace=True)
fold_change_nrp1.fillna(0,inplace=True)

cutoff=8

fig=plt.figure(figsize=(10,30))
grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.2)

ax = plt.subplot(grid[0,0])
ax1 = plt.subplot(grid[1,0])
ax2 = plt.subplot(grid[2,0])

ax.set_title('Potential Negative Interactors for nrp1')
ax.plot(fold_change,alpha=0.6,color='red')
ax.hlines(y=cutoff,xmin=0,xmax=14000,linestyles='--',alpha=0.3,label='cutoff')

ax.set_ylabel('fold change reads per tr')
ax.set_ylim(0,100)
## annotated centromeres
for i in np.arange(0,len(data_wt)):
    
    
    if fold_change[i]>cutoff and data_wt.loc[i,'Standard_name']!='noncoding':
        ax.vlines(x=i,ymin=0,ymax=100,linestyles='-',alpha=0.2)
        ax.text(x=i,y=60,s=data_wt.loc[i,'Standard_name'],rotation=90,fontsize=8)
    if fold_change[i]>cutoff and data_wt.loc[i,'Essentiality']==1 :
        ax.vlines(x=i,ymin=0,ymax=100,linestyles='-',alpha=0.2)
        ax.text(x=i,y=60,s=data_wt.loc[i,'Standard_name'],rotation=90,fontsize=8,color='red')

ax1.set_title('Potential Positive Interactors for nrp1')
ax1.plot(fold_change_nrp1,alpha=0.6,color='green')
ax1.hlines(y=cutoff,xmin=0,xmax=14000,linestyles='--',alpha=0.3,label='cutoff')
ax1.set_xlabel('genomic regions')
ax1.set_ylabel('fold change reads per tr')
ax1.set_ylim(0,100)
## annotated centromeres
for i in np.arange(0,len(data_wt)):
    
    
    if fold_change_nrp1[i]>cutoff and data_wt.loc[i,'Standard_name']!='noncoding' :
        ax1.vlines(x=i,ymin=0,ymax=100,linestyles='-',alpha=0.2)
        ax1.text(x=i,y=60,s=data_wt.loc[i,'Standard_name'],rotation=90,fontsize=8)
    if fold_change_nrp1[i]>cutoff and data_wt.loc[i,'Essentiality']==1 :
        ax1.vlines(x=i,ymin=0,ymax=100,linestyles='-',alpha=0.2)
        ax1.text(x=i,y=60,s=data_wt.loc[i,'Standard_name'],rotation=90,fontsize=8,color='red')
        
ax.legend()
ax1.legend()

ax2.plot(data_wt['reads-per-tr'],data_nrp1['reads-per-tr'],'o',color='gray')
ax2.set_xlim(0,3000)
ax2.set_ylim(0,3000)
ax2.set_xlabel('reads per tr WT')
ax2.set_ylabel('reads per tr dnrp1')